# ReadTrack: School Reading Challenge

The database schema is in the instruction Google Docs. Run the below code to create a new db file.

In [33]:
# Run to link to db and attempt to unlock db when needed.
import sqlite3
import csv

# !NOTE: if db file does not exist, it will be created
db_file = "readtrack.db"
conn = sqlite3.connect(db_file)
conn.close()

The below SQLite Code will create the tables in `readtrack.db`.

In [34]:
query1 = '''
CREATE TABLE IF NOT EXISTS Member (
    MemberID      TEXT PRIMARY KEY,
    Class         TEXT NOT NULL,
    IndexNo       INTEGER NOT NULL CHECK (IndexNo BETWEEN 1 AND 40),
    Gender        TEXT NOT NULL CHECK (Gender IN ('M','F')),
    YearEnrolled  INTEGER NOT NULL CHECK (YearEnrolled BETWEEN 2019 AND 2025)
);
'''
query2 = '''
CREATE TABLE IF NOT EXISTS Book (
    BookID   INTEGER PRIMARY KEY,
    Title    TEXT NOT NULL UNIQUE,
    Author   TEXT NOT NULL,
    Genre    TEXT NOT NULL CHECK (Genre IN ('Fantasy','Sci-Fi','Mystery','Non-Fiction','Historical','Horror'))
);
'''
query3 = '''
CREATE TABLE IF NOT EXISTS ReadingLog (
    MemberID       TEXT NOT NULL,
    BookID         INTEGER NOT NULL,
    DateCompleted  TEXT NOT NULL CHECK (length(DateCompleted)=10),
    PRIMARY KEY (MemberID, BookID),
    FOREIGN KEY (MemberID) REFERENCES Member(MemberID) ON DELETE CASCADE ON UPDATE CASCADE,
    FOREIGN KEY (BookID)   REFERENCES Book(BookID)   ON DELETE CASCADE ON UPDATE CASCADE
);
'''
queries = [query1, query2, query3]

In [35]:
conn = sqlite3.connect(db_file)
for query in queries:
    conn.execute(query)
conn.commit()
conn.close()

#### Task 1
Read data from the csv files in the `data_files` folder, and insert the data into the respective tables.

In [36]:
# Task 1 Answer:

files = {
    'Book': 'books.csv',
    'Member': 'members.csv',
    'ReadingLog': 'reading_logs.csv'
}

for file in files:
    with open(f'data_files/{files[file]}', 'r') as f:
        # print(file, files[file]) # table name, file name
        csv_reader = csv.reader(f)
        header = next(csv_reader)
        # print(header)
        data = []
        for row in csv_reader:
            ph = ",".join(['?' for _ in header])
            # print(header, ph)
            conn = sqlite3.connect('readtrack.db')
            query2 = f"""
            INSERT OR IGNORE INTO {file}
            ({','.join(header)})
            VALUES
            ({ph})
            """
            # print(','.join(header))
            try:
                conn.execute(query2, tuple(row))
            except sqlite3.Error as e:
                print(e)
            conn.commit()
            conn.close()